In [1]:
import pandas as pd

# Load the datasets (replace 'path_to_file' with actual file paths if running locally)
og_data = pd.read_csv('og-unclean-data.csv')
spatial_data = pd.read_csv('spatial-unclean-data.csv')
temporal_data = pd.read_csv('temporal-unclean-data.csv')

# Step 1: Drop irrelevant columns
og_data_cleaned = og_data.drop(columns=['Unnamed: 13', 'Unnamed: 14'])
spatial_data_cleaned = spatial_data.drop(columns=['Unnamed: 3'])
temporal_data_cleaned = temporal_data.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

# Step 2: Handle missing values
# Forward fill for consistency
og_data_cleaned.fillna(method='ffill', inplace=True)

# Drop rows where MPA or FisheryConsumption is missing in spatial and temporal datasets
spatial_data_cleaned.dropna(subset=['MPA', 'Fishery'], inplace=True)
temporal_data_cleaned.dropna(subset=['MPA', 'FisheryConsumption'], inplace=True)

# The cleaned datasets
cleaned_og_data = og_data_cleaned
cleaned_spatial_data = spatial_data_cleaned
cleaned_temporal_data = temporal_data_cleaned

# Save the cleaned datasets to CSV files

# Paths for saving cleaned data
cleaned_og_data_path = 'cleaned_og_data.csv'
cleaned_spatial_data_path = 'cleaned_spatial_data.csv'
cleaned_temporal_data_path = 'cleaned_temporal_data.csv'

# Save the cleaned data
cleaned_og_data.to_csv(cleaned_og_data_path, index=False)
cleaned_spatial_data.to_csv(cleaned_spatial_data_path, index=False)
cleaned_temporal_data.to_csv(cleaned_temporal_data_path, index=False)

cleaned_og_data_path, cleaned_spatial_data_path, cleaned_temporal_data_path


# Output cleaned data to confirm the result
cleaned_og_data.head(), cleaned_spatial_data.head(), cleaned_temporal_data.head()


/var/folders/g9/s_fx5qk126515rbzxjm9x6ww0000gn/T/ipykernel_66268/2419637042.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  og_data_cleaned.fillna(method='ffill', inplace=True)


(                  Country Name        2017        2018        2019  \
 0                        Aruba      149.50      150.00      164.50   
 1  Africa Eastern and Southern  3897511.90  3883753.45  3821770.86   
 2                  Afghanistan     9000.00    10000.00    10729.00   
 3   Africa Western and Central  3706566.44  3642143.61  3667626.70   
 4                       Angola   532914.00   444858.00   401696.00   
 
          2020        2021        2022    2017.1    2018.1    2019.1    2020.1  \
 0      161.50      166.50      173.50  0.000105  0.000105  0.000105  0.000105   
 1  3805060.99  4097350.48  3725334.12  4.945690  4.945687  5.235880  5.036564   
 2    11444.00    12907.00    13150.00  4.945690  4.945687  5.235880  5.036564   
 3  3523165.88  3701624.19  3894655.40  4.945690  4.945687  5.235880  5.036564   
 4   376515.00   531522.00   471672.00  0.004930  0.004930  0.004930  0.004930   
 
      2021.1    2022.1  
 0  0.000105  0.000105  
 1  5.040512  0.000105  
 2 

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Load the datasets into pandas DataFrames
og_data_cleaned = pd.read_csv(cleaned_og_data_path)
spatial_data_cleaned = pd.read_csv(cleaned_spatial_data_path)
temporal_data_cleaned = pd.read_csv(cleaned_temporal_data_path)

categorical_columns = ['Country Name']

# Encoding categorical columns in og_data_cleaned
for col in categorical_columns:
    og_data_cleaned[col] = label_encoder.fit_transform(og_data_cleaned[col])

# Encoding categorical columns in spatial_data_cleaned
for col in categorical_columns:
    spatial_data_cleaned[col] = label_encoder.fit_transform(spatial_data_cleaned[col])


# The cleaned and encoded datasets
cleaned_og_data = og_data_cleaned
cleaned_spatial_data = spatial_data_cleaned
cleaned_temporal_data = temporal_data_cleaned

# Paths for saving cleaned data
cleaned_og_data_path = 'cleaned_encoded_og_data.csv'
cleaned_spatial_data_path = 'cleaned_encoded_spatial_data.csv'
cleaned_temporal_data_path = 'cleaned_encoded_temporal_data.csv'

# Save the cleaned and encoded data
cleaned_og_data.to_csv(cleaned_og_data_path, index=False)
cleaned_spatial_data.to_csv(cleaned_spatial_data_path, index=False)
cleaned_temporal_data.to_csv(cleaned_temporal_data_path, index=False)

In [3]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

# Define file paths
og_file_path = 'cleaned_encoded_og_data.csv'
spatial_file_path = 'cleaned_encoded_spatial_data.csv'
temporal_file_path = 'cleaned_encoded_temporal_data.csv'

# Load the datasets into pandas DataFrames
og_data = pd.read_csv(og_file_path)
spatial_data = pd.read_csv(spatial_file_path)
temporal_data = pd.read_csv(temporal_file_path)

datasets = {
    'og_data': og_data,
    'spatial_data': spatial_data,
    'temporal_data': temporal_data
}

for name, data in datasets.items(): # Iterate using name and data
  # One-Hot Encoding for categorical columns
  categorical_columns = ['Country Name']
  if 'Country Name' in data.columns: # Check if column exists before encoding
    encoder = OneHotEncoder(sparse=False)
    encoded_categories = encoder.fit_transform(data[categorical_columns])

    # Create a DataFrame with the encoded categorical columns
    encoded_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out(categorical_columns))

    # Drop the original categorical columns and concatenate the encoded columns
    data = data.drop(categorical_columns, axis=1)
    data = pd.concat([data, encoded_df], axis=1)

  # Normalization
  scaler = MinMaxScaler()
  normalized_data = scaler.fit_transform(data)

  # Standardization
  standardizer = StandardScaler()
  standardized_data = standardizer.fit_transform(data)

  # Convert the numpy arrays back to DataFrames
  normalized_df = pd.DataFrame(normalized_data, columns=data.columns)
  standardized_df = pd.DataFrame(standardized_data, columns=data.columns)

  # Save the transformed data to new CSV files, using the dataset name in the filename
  normalized_df.to_csv(f'{name}-nrm.csv', index=False)
  standardized_df.to_csv(f'{name}-std.csv', index=False)

  # Display the first few rows of each DataFrame
  print(f"Encoded and Normalized {name} Data:")
  print(normalized_df.head())

  print(f"\nEncoded and Standardized {name} Data:")
  print(standardized_df.head())


/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Encoded and Normalized og_data Data:
           2017          2018          2019          2020          2021  \
0  4.465208e-07  4.116628e-07  4.329412e-07  8.553970e-08  1.650334e-07   
1  1.901961e-02  1.837668e-02  1.808274e-02  1.795210e-02  1.888238e-02   
2  4.363704e-05  4.701946e-05  5.041991e-05  5.331784e-05  5.888069e-05   
3  1.808779e-02  1.723344e-02  1.735339e-02  1.662208e-02  1.705864e-02   
4  2.600342e-03  2.104662e-03  1.900319e-03  1.775770e-03  2.448961e-03   

           2022    2017.1    2018.1    2019.1    2020.1  ...  \
0  3.979066e-08  0.000001  0.000001  0.000001  0.000001  ...   
1  1.743846e-02  0.049457  0.049540  0.052359  0.050366  ...   
2  6.078609e-05  0.049457  0.049540  0.052359  0.050366  ...   
3  1.823110e-02  0.049457  0.049540  0.052359  0.050366  ...   
4  2.207244e-03  0.000049  0.000049  0.000049  0.000049  ...   

   Country Name_256  Country Name_257  Country Name_258  Country Name_259  \
0               0.0               0.0             